In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numba

In [2]:
def b_rules(lasR, LasR, lasI, LasI, AI1, LasRAI1):
    
    k_lasR = 0.004
    g_lasR = 0.002
    k_LasR = 0.4
    g_LasR = 0.35
    a_lasI = 0.00036
    b_lasI = 0.0058
    K1 = 1.2
    h1 = -1.4
    g_lasI = 0.001
    k_LasI = 0.7
    g_LasI = 0.12
    k_AI1 = 1
    g_AI1 = 0.3
    u_LasRAI1 = 0.05
    s_LasRAI1 = 0.25
    g_LasRAI1 = 0.14
    # Cell parameters
    d = 0.8
    
    S0 = k_lasR                                       # + lasR
    S1 = lasR*g_lasR                                  # - lasR
    S2 = lasR*k_LasR                                  # + LasR
    S3 = LasRAI1*s_LasRAI1                            # + LasR ; + AI1 ; - LasRAI1
    S4 = AI1*LasR*u_LasRAI1                           # - LasR ; - AI1 ; + LasRAI1
    S5 = LasR*g_LasR                                  # - LasR
    S6 = a_lasI + (b_lasI/(1+(LasRAI1/K1)**h1))       # + lasI
    S7 = lasI*g_lasI                                  # - lasI
    S8 = lasI*k_LasI                                  # + LasI
    S9 = LasI*g_LasI                                  # - LasI
    S10 = LasI*k_AI1                                  # + AI1
    S11 = AI1*g_AI1                                   # - AI1
    S12 = d*AI1                                       # - AI1 ; + AI1_ext
    S13 = LasRAI1*g_LasRAI1                           # - LasRAI1
    
    Stot = S0 + S1 + S2 + S3 + S4 + S5 + S6 + S7 + S8 + S9 + S10 + S11 + S12 + S13
    Tau = np.random.exponential(scale=1/Stot)
    U = np.random.rand()
    
    if (U < S0/Stot):
        return Tau, 0
    elif (U < (S0+S1)/Stot):
        return Tau, 1
    elif (U < (S1+S2+S3)/Stot):
        return Tau, 2
    elif (U <(S0+S1+S2+S3)/Stot):
        return Tau, 3
    elif (U <(S0+S1+S2+S3+S4)/Stot):
        return Tau, 4
    elif (U <(S0+S1+S2+S3+S4+S5)/Stot):
        return Tau, 5
    elif (U <(S0+S1+S2+S3+S4+S5+S6)/Stot):
        return Tau, 6
    elif (U <(S0+S1+S2+S3+S4+S5+S6+S7)/Stot):
        return Tau, 7
    elif (U <(S0+S1+S2+S3+S4+S5+S6+S7+S8)/Stot):
        return Tau, 8
    elif (U <(S0+S1+S2+S3+S4+S5+S6+S7+S8+S9)/Stot):
        return Tau, 9
    elif (U <(S0+S1+S2+S3+S4+S5+S6+S7+S8+S9+S10)/Stot):
        return Tau, 10
    elif (U <(S0+S1+S2+S3+S4+S5+S6+S7+S8+S9+S10+S11)/Stot):
        return Tau, 11
    elif (U <(S0+S1+S2+S3+S4+S5+S6+S7+S8+S9+S10+S11+S12)/Stot):
        return Tau, 12
    else:
        return Tau, 13

In [3]:
def e_rules(AI1_ext):
    
    g_AI1_ext = 0.8
    d = 0.8
    d_away = 1.2
    
    S14 = AI1_ext*d
    S15 = AI1_ext*(d_away + g_AI1_ext)
    
    Stot = S14 + S15
    Tau = np.random.exponential(scale=1/Stot)
    U = np.random.rand()
    
    if (U < S14/Stot):
        return Tau, 14
    else:
        return Tau, 15

In [4]:
b_0 = np.array((0, 0, 0, 0, 0, 0))   # (lasR_0, LasR_0, lasI_0, LasI_0, AI1_0, LasRAI1_0)
e_0 = np.array((0))                  # (AI1_ext_0)
def multicompartmental_gillespie(N, steps, b_0, e_0):
    
    bacteria = np.zeros(N, len(b_0), steps)
    while i < N:
        while j < len(b_0):
            bacteria[i][j][0] = b_0[j]
            j += 1
        i +=1

    env = np.zeros(len(e_0), steps)
    while i < len(e_0):
        env[i][0] = e_0[i]
    
    while i < steps:
        reaction = np.zeros(N, 2)
        while j < N:
            reaction[j][0], reaction[j][1] = b_rules(bacteria[j][:][i-1])